In [1]:
%load_ext autoreload
%autoreload 2
import json
import os
import sys



import keras.backend as K
from keras.models import load_model
from keras.optimizers import Adam
import numpy as np
import tensorflow as tf

from dataset_utils import adni_loader
#from networks import transform_network_utils

# sys.path.append('../neuron')
# sys.path.append('../voxelmorph/src')
# from dense_3D_spatial_transformer import Dense3DSpatialTransformer
# sys.path.append('../voxelmorph-sandbox/')
# import voxelmorph.losses as vm_losses

gpu_ids = [0]
# set gpu id and tf settings
os.environ['CUDA_VISIBLE_DEVICES']=','.join([str(g) for g in gpu_ids])
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.tensorflow_backend.set_session(tf.Session(config=config))



Using TensorFlow backend.
../cnn_utils/vis_utils.py:14: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  mpl.use('Agg')


Load Segmenter experiment

In [2]:
import json
import os
import sys

sys.path.append('../cnn_utils')
import file_utils
from keras.models import load_model

from experiments_VTE import FewShotSegmentationExperimentClass

# load pretrained segmenter network
seg_exp_dir = './experiments/FewShotSeg_adni-unnorm-masked_100ul_subj-OASIS_OAS1_0327-l_ims160-192-1_arch32-32-64-64-128-128_vteaug-gen-unets_seq_bidir_test_wrapper-colorregwt1-e1010'
#seg_exp_dir = './experiments/FewShotSeg_adni-unnorm-masked_100ul_subj-OASIS_OAS1_0327-l_ims160-192-1_arch32-32-64-64-128-128_augflow-amp200_blur12_randmult0.5_flowaug-gen'

with open(os.path.join(seg_exp_dir, 'arch_params.json'), 'r') as f:
	arch_params = json.load(f)
with open(os.path.join(seg_exp_dir, 'data_params.json'), 'r') as f:
	data_params = json.load(f)
    
seg_exp = FewShotSegmentationExperimentClass.ExperimentSegmenter(arch_params=arch_params, data_params=data_params)
model_name = seg_exp.get_model_name()

model_name, exp_dir, figures_dir, logs_dir, models_dir = file_utils.make_output_dirs(model_name, prompt_delete=False)
seg_exp.save_exp_info(exp_dir, figures_dir, models_dir, logs_dir)

seg_exp.load_data(load_fewer=True)

# create models and load existing ones if necessary
seg_exp.create_models()

# run our segmenter on the test set so we can look at test metrics
seg_exp.create_generators(batch_size=16)

# TODO: parse training logs for early stopping
loaded_epoch = seg_exp.load_models(models_dir, 'latest') - 1


Setting up GLT class from dir experiments/GLT_adni-unnorm-masked_100ul_subj-OASIS_OAS1_0327-l-to-subjs_ims160-192-224-1_voxelmorphbidir_flow_color_colorsrcspace_lr0.0005_pt1000_frzflow_grad_l2_regfwt1_cc_vm_wt1_win9_grad-seg-l2_regcwt1_l2_sigI0.1
[2018-11-09 09:39:38] Loading adni dataset adni-unnorm-masked_100ul_subj-OASIS_OAS1_0327-l
[2018-11-09 09:39:38] Params: {'dataset_name': 'adni', 'source_name': 'centroidsubj2', 'target_name': 'subjs', 'unnormalized': True, 'masked': True, 'n_shot': 1, 'use_atlas_as_source': False, 'use_subject': 'OASIS_OAS1_0327_MR1_mri_talairach_orig', 'img_shape': [160, 192, 224, 1], 'pred_img_shape': [160, 192, 1], 'aug_img_shape': [160, 192, 224, 1], 'n_unlabeled': 100, 'n_validation': 50, 'load_vols': False, 'aug_in_gen': True, 'n_vte_aug': None, 'n_flow_aug': None, 'use_labels': [0, 16, 10, 49, 8, 47, 4, 43, 7, 46, 12, 51, 2, 41, 28, 60, 11, 50, 13, 52, 17, 53, 14, 15, 18, 54, 24, 3, 42, 31, 63], 'final_test': False, 'warp_labels': True, 'n_dims': 3, 's

../cnn_utils/vis_utils.py:14: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  mpl.use('Agg')


Got list of 7329 files from /data/ddmg/voxelmorph/data/t1_mix/proc/resize256-crop_x32/train/origs/*.npz:
ADNI_ADNI-3T-FS-5.3-Long_293689.long.016_S_4591_base_mri_talairach_orig.npz
ADNI_ADNI-3T-FS-5.3-Long_78841.long.016_S_1326_base_mri_talairach_orig.npz
ADNI_ADNI-1.5T-FS-5.3-Long_436815.long.094_S_1330_base_mri_talairach_orig.npz
ADNI_ADNI-3T-FS-5.3-Long_296323.long.068_S_2168_base_mri_talairach_orig.npz
ADNI_ADNI-3T-FS-5.3-Long_388923.long.135_S_5273_base_mri_talairach_orig.npz
ADNI_ADNI-3T-FS-5.3-Long_272700.long.009_S_4388_base_mri_talairach_orig.npz
ADNI_ADNI-1.5T-FS-5.3-Long_394785.long.027_S_0408_base_mri_talairach_orig.npz
PPMI_3519_mri_talairach_orig.npz
ADNI_ADNI-3T-FS-5.3-Long_119158.long.053_S_0507_base_mri_talairach_orig.npz
ADNI_ADNI-1.5T-FS-5.3-Long_63306.long.007_S_0249_base_mri_talairach_orig.npz
ABIDE_50685_mri_talairach_orig.npz
ADNI_ADNI-1.5T-FS-5.3-Long_121666.long.041_S_1423_base_mri_talairach_orig.npz
GSP_120719_TT88SP_FS_mri_talairach_orig.npz
COBRE_0040043_mri

vm_bidir_bck_model
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 160, 192, 224 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 160, 192, 224 0                                            
__________________________________________________________________________________________________
concatenate_6 (Concatenate)     (None, 160, 192, 224 0           input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
conv3d_12 (Conv3D)              (None, 80, 96, 112,  880         concatenate_6[0][0]      

[2018-11-09 09:39:48] Layer (type)                    Output Shape         Param #     Connected to                     
[2018-11-09 09:39:48] ==================================================================================================
[2018-11-09 09:39:48] img_input_src (InputLayer)      (None, 160, 192, 224 0                                            
[2018-11-09 09:39:48] __________________________________________________________________________________________________
[2018-11-09 09:39:48] img_input_flowtgt (InputLayer)  (None, 160, 192, 224 0                                            
[2018-11-09 09:39:48] __________________________________________________________________________________________________
[2018-11-09 09:39:48] vm_bidir_bck_model (Model)      [(None, 160, 192, 22 300547      img_input_flowtgt[0][0]          
[2018-11-09 09:39:48]                                                                  img_input_src[0][0]              
[2018-11-09 09:39:48] __________

...

[2018-11-09 09:39:50] Looking for models in ./experiments/GLT_adni-unnorm-masked_100ul_subj-OASIS_OAS1_0327-l-to-subjs_ims160-192-224-1_voxelmorphbidir_flow_color_colorsrcspace_lr0.0005_pt1000_frzflow_grad_l2_regfwt1_cc_vm_wt1_win9_grad-seg-l2_regcwt1_l2_sigI0.1/models
[2018-11-09 09:39:50] Loading model vm_bidir_fwd_model from ./experiments/GLT_adni-unnorm-masked_100ul_subj-OASIS_OAS1_0327-l-to-subjs_ims160-192-224-1_voxelmorphbidir_flow_color_colorsrcspace_lr0.0005_pt1000_frzflow_grad_l2_regfwt1_cc_vm_wt1_win9_grad-seg-l2_regcwt1_l2_sigI0.1/models/vm_bidir_fwd_model_epoch1010_iter101000.h5
[2018-11-09 09:39:50] Loading model vm_bidir_bck_model from ./experiments/GLT_adni-unnorm-masked_100ul_subj-OASIS_OAS1_0327-l-to-subjs_ims160-192-224-1_voxelmorphbidir_flow_color_colorsrcspace_lr0.0005_pt1000_frzflow_grad_l2_regfwt1_cc_vm_wt1_win9_grad-seg-l2_regcwt1_l2_sigI0.1/models/vm_bidir_bck_model_epoch1010_iter101000.h5
[2018-11-09 09:39:50] Loading model color_delta_unet_srcspace from ./exp

[2018-11-09 09:39:54] __________________________________________________________________________________________________
[2018-11-09 09:39:54] leaky_re_lu_44 (LeakyReLU)      (None, 40, 48, 32)   0           unet_enc_conv2D_2_2[0][0]        
[2018-11-09 09:39:54] __________________________________________________________________________________________________
[2018-11-09 09:39:54] unet_enc_maxpool_2 (MaxPooling2 (None, 20, 24, 32)   0           leaky_re_lu_44[0][0]             
[2018-11-09 09:39:54] __________________________________________________________________________________________________
[2018-11-09 09:39:54] unet_enc_conv2D_3_1 (Conv2D)    (None, 20, 24, 64)   18496       unet_enc_maxpool_2[0][0]         
[2018-11-09 09:39:54] __________________________________________________________________________________________________
[2018-11-09 09:39:54] leaky_re_lu_45 (LeakyReLU)      (None, 20, 24, 64)   0           unet_enc_conv2D_3_1[0][0]        
[2018-11-09 09:39:54] __________

[2018-11-09 09:39:54] __________________________________________________________________________________________________
[2018-11-09 09:39:54] leaky_re_lu_57 (LeakyReLU)      (None, 10, 12, 128)  0           unet_dec_conv2D_1_0[0][0]        
[2018-11-09 09:39:54] __________________________________________________________________________________________________
[2018-11-09 09:39:54] unet_dec_conv2D_1_1 (Conv2D)    (None, 10, 12, 128)  147584      leaky_re_lu_57[0][0]             
[2018-11-09 09:39:54] __________________________________________________________________________________________________
[2018-11-09 09:39:54] leaky_re_lu_58 (LeakyReLU)      (None, 10, 12, 128)  0           unet_dec_conv2D_1_1[0][0]        
[2018-11-09 09:39:54] __________________________________________________________________________________________________
[2018-11-09 09:39:54] unet_dec_upsamp_2 (UpSampling2D (None, 20, 24, 128)  0           leaky_re_lu_58[0][0]             
[2018-11-09 09:39:54] __________

[2018-11-09 09:39:54]                                                                  leaky_re_lu_40[0][0]             
[2018-11-09 09:39:54] __________________________________________________________________________________________________
[2018-11-09 09:39:54] unet_dec_conv2D_last_0 (Conv2D) (None, 160, 192, 31) 9238        concatenate_23[0][0]             
[2018-11-09 09:39:54] __________________________________________________________________________________________________
[2018-11-09 09:39:54] leaky_re_lu_71 (LeakyReLU)      (None, 160, 192, 31) 0           unet_dec_conv2D_last_0[0][0]     
[2018-11-09 09:39:54] __________________________________________________________________________________________________
[2018-11-09 09:39:54] unet_dec_conv2D_final (Conv2D)  (None, 160, 192, 31) 992         leaky_re_lu_71[0][0]             
[2018-11-09 09:39:54] __________________________________________________________________________________________________
[2018-11-09 09:39:54] activation

[2018-11-09 09:39:55] Augmenting by vte in the generator!
[2018-11-09 09:39:55] Found latest epoch 3800 in dir ./experiments/FewShotSeg_adni-unnorm-masked_100ul_subj-OASIS_OAS1_0327-l_ims160-192-1_arch32-32-64-64-128-128_vteaug-gen-unets_seq_bidir_test_wrapper-colorregwt1-e1010/models
[2018-11-09 09:39:55] Looking for models in ./experiments/FewShotSeg_adni-unnorm-masked_100ul_subj-OASIS_OAS1_0327-l_ims160-192-1_arch32-32-64-64-128-128_vteaug-gen-unets_seq_bidir_test_wrapper-colorregwt1-e1010/models
[2018-11-09 09:39:55] Loading model segmenter_unet from ./experiments/FewShotSeg_adni-unnorm-masked_100ul_subj-OASIS_OAS1_0327-l_ims160-192-1_arch32-32-64-64-128-128_vteaug-gen-unets_seq_bidir_test_wrapper-colorregwt1-e1010/models/segmenter_unet_epoch3800_iter26600.h5


In [3]:
import vte_runner
test_data_params = vte_runner.named_vte_data_params['adni-csts2-test']
print(test_data_params)
test_ds = adni_loader.ADNIDataset(test_data_params)
_ = test_ds.load_dataset()

{'dataset_name': 'adni', 'source_name': 'centroidsubj2', 'target_name': 'subjs', 'final_test': True, 'n_test': 100, 'unnormalized': True, 'masked': True, 'n_shot': 1, 'use_atlas_as_source': False, 'use_subject': 'OASIS_OAS1_0327_MR1_mri_talairach_orig', 'img_shape': (160, 192, 224, 1), 'pred_img_shape': (160, 192, 1), 'aug_img_shape': (160, 192, 224, 1), 'n_unlabeled': 1, 'n_validation': 1, 'load_vols': True, 'aug_in_gen': True, 'n_vte_aug': None, 'n_flow_aug': None, 'use_labels': [0, 16, 10, 49, 8, 47, 4, 43, 7, 46, 12, 51, 2, 41, 28, 60, 11, 50, 13, 52, 17, 53, 14, 15, 18, 54, 24, 3, 42, 31, 63], 'warp_labels': True, 'n_dims': 3}
Loading adni dataset adni-unnorm-masked_1ul_subj-OASIS_OAS1_0327-l
Params: {'dataset_name': 'adni', 'source_name': 'centroidsubj2', 'target_name': 'subjs', 'final_test': True, 'n_test': 100, 'unnormalized': True, 'masked': True, 'n_shot': 1, 'use_atlas_as_source': False, 'use_subject': 'OASIS_OAS1_0327_MR1_mri_talairach_orig', 'img_shape': (160, 192, 224, 1)

In [4]:

# show what our labeled example looks like, as a sanity check
import IPython
import PIL
source_X = seg_exp.dataset.X_labeled_train#adni_loader._load_vol_and_seg(ds.files_labeled_train[0], load_seg=False, mask_vol=ds.params['masked'])
source_Y = seg_exp.dataset.Y_labeled_train
print(seg_exp.dataset.files_labeled_train)
print(seg_exp.dataset.files_labeled_test[:10])
IPython.display.display(PIL.Image.fromarray((source_X[0, :, :, 64, 0]*255).astype(np.uint8)))
IPython.display.display(PIL.Image.fromarray((seg_exp.dataset.X_labeled_test[0, :, :, 64, 0]*255).astype(np.uint8)))
IPython.display.display(PIL.Image.fromarray((seg_exp.dataset.Y_labeled_test[0, :, :, 64]*255).astype(np.uint8)))
print(test_ds.X_labeled_test.shape)
print(np.any(np.isnan(test_ds.X_labeled_test)))
print(np.any(np.isnan(test_ds.Y_labeled_test)))

['/data/ddmg/voxelmorph/data/t1_mix/proc/resize256-crop_x32/train/origs/OASIS_OAS1_0327_MR1_mri_talairach_orig.npz', '/data/ddmg/voxelmorph/data/t1_mix/proc/resize256-crop_x32/train/origs/OASIS_OAS1_0327_MR1_mri_talairach_orig.npz']


AttributeError: 'ADNIDataset' object has no attribute 'files_labeled_test'

In [5]:

seg_exp.dataset = test_ds
print(seg_exp.dataset.params)

# one-time thing because we forgot to add this to the data loader
# test_labels = np.unique(test_ds.Y_labeled_test)
# for l in test_labels:
#     if l not in seg_exp.label_mapping:
#         test_ds.Y_labeled_test[test_ds.Y_labeled_test == l] = 0

do_final_test = True
seg_exp.dataset.logger = seg_exp.logger
if do_final_test:
    final_test_gen = seg_exp.dataset.gen_vols_batch(
            ['labeled_test'], batch_size=1, randomize=False,
    )

    n_test_examples = seg_exp.dataset.params['n_test']
    test_cces, test_dice, test_accs = seg_exp._eval_from_gen(final_test_gen, n_test_examples)
else:
    valid_gen = seg_exp.dataset.gen_vols_batch(
            ['labeled_valid'], batch_size=1, randomize=False,
    )

    n_valid_examples = seg_exp.dataset.params['n_valid']
    valid_cces, valid_dice, valid_accs = seg_exp._eval_from_gen(valid_gen, n_valid_examples)




{'dataset_name': 'adni', 'source_name': 'centroidsubj2', 'target_name': 'subjs', 'final_test': True, 'n_test': 100, 'unnormalized': True, 'masked': True, 'n_shot': 1, 'use_atlas_as_source': False, 'use_subject': 'OASIS_OAS1_0327_MR1_mri_talairach_orig', 'img_shape': (160, 192, 224, 1), 'pred_img_shape': (160, 192, 1), 'aug_img_shape': (160, 192, 224, 1), 'n_unlabeled': 1, 'n_validation': 1, 'load_vols': True, 'aug_in_gen': True, 'n_vte_aug': None, 'n_flow_aug': None, 'use_labels': [0, 16, 10, 49, 8, 47, 4, 43, 7, 46, 12, 51, 2, 41, 28, 60, 11, 50, 13, 52, 17, 53, 14, 15, 18, 54, 24, 3, 42, 31, 63], 'warp_labels': True, 'n_dims': 3, 'orig_img_shape': (160, 192, 224, 1), 'scale': 1.0, 'split_id': None}
[2018-11-09 09:44:00] Testing on subject 0 of 100
[2018-11-09 09:44:00] LOOKING FOR FINAL TEST SET
0.8996213769054576
[2018-11-09 09:44:11] Testing on subject 1 of 100
0.8967939890913752
[2018-11-09 09:44:17] Testing on subject 2 of 100
0.894708044669655
[2018-11-09 09:44:23] Testing on su

[2018-11-09 09:54:10] Mean dice (no bkg): 0.7594273449925036
[2018-11-09 09:54:10] Mean CE: 0.10099492590310097
[2018-11-09 09:54:10] Mean accuracy: 0.8996200147889578


In [ ]:
# save our test results in a mat file so that we can parse them later

In [6]:
results_dir = './segmentation_test_results'
if not os.path.isdir(results_dir):
    os.mkdir(results_dir)
results_file = os.path.join(results_dir, seg_exp.model_name + '_e{}.mat'.format(loaded_epoch))
import scipy.io as sio
sio.savemat(results_file, {
    'cce': test_cces,
    'dice': test_dice,
    'acc': test_accs
    
})

## Detailed analysis of segmentations including dice with fully-supervised predictions

In [ ]:
sys.path.append('../voxelmorph-sandbox/')
import voxelmorph.losses as vm_losses

seg_sup_model_file = './experiments/FewShotSeg_adni-unnorm-masked_1ul_100l_ims160-192-1_arch32-32-64-64-128-128/models/segmenter_unet_epoch6000_iter42000.h5'
dice_loss = vm_losses.diceLoss

seg_sup_model = load_model(seg_sup_model_file, custom_objects={'dice_loss': dice_loss}, compile=False)

# Load GLT experiment to test voxelmorph model

# glt_dir = 'experiments/GLT_adni-unnorm-masked_100ul_atlas-l-to-subjs_ims160-192-224-1_voxelmorph2guha_flow_color_unet16-32-32-32_dec64-64-32-32-32-32-32_lr0.0005_pt1000_frzflow_grad_l2_regfwt1_cc_vm_wt1_win9_grad-seg-l2_regcwt0.5_l2_sigI0.1'
# glt_dir = 'experiments/GLT_adni-unnorm-masked_100ul_atlas-l-to-subjs_ims160-192-224-1_voxelmorph2guha_flow_color_unet16-32-32-32_dec64-64-32-32-32-32-32_lr0.0005_pt1000_frzflow_grad_l2_regfwt1_cc_vm_wt1_win9_grad-seg-l2_regcwt1_l2_sigI0.1'

# with open(os.path.join(glt_dir, 'arch_params.json'), 'r') as f:
# 	arch_params = json.load(f)
# with open(os.path.join(glt_dir, 'data_params.json'), 'r') as f:
# 	data_params = json.load(f)



# from experiments_VTE import GLTExperimentClass	
# glt_exp = GLTExperimentClass.ExperimentGlobalLocalTransforms(data_params, arch_params)
# model_name = glt_exp.get_model_name()

# model_name, exp_dir, figures_dir, logs_dir, models_dir = file_utils.make_output_dirs(model_name, prompt_delete=False)
# glt_exp.save_exp_info(exp_dir, figures_dir, models_dir, logs_dir)

# glt_exp.dataset.params['load_vols'] = False
# glt_exp.dataset.params['warp_labels'] = False
# glt_exp.load_data(load_fewer=True)

# # create models and load existing ones if necessary
# glt_exp.create_models()


In [ ]:
vm_model = seg_exp.voxelmorph_model
# glt_exp.load_models(models_dir, 1100)
# glt_exp.compile_models()
# # load pre-trained voxelmorph model?
# sys.path.append('../voxelmorph-sandbox')
# from voxelmorph.dense_3D_spatial_transformer import Dense3DSpatialTransformer
# start_iter = 0
# vm_model = load_model(
#     #'/afs/csail.mit.edu/u/x/xamyzhao/voxelmorph/models/vm2_cc.h5',
#     './experiments/voxelmorph/vm2_cc_AtoUMS_newdataset_iter100000_wrapper.h5',#.format(start_iter),
#     custom_objects={'Dense3DSpatialTransformer': Dense3DSpatialTransformer, 'tf': tf},
#     compile=False,
# )

#vm_model = glt_exp.tester_model

vm_model.summary()



In [ ]:
import vis_utils
import medipy_metrics
from networks import transform_network_utils
from networks import segmenter_networks
import classification_utils

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important;  }</style>"))
n_examples = 20


train_vol_gen = ds.gen_vols_batch(['unlabeled_train'], batch_size=1, randomize=False, load_segs=True)
test_vol_gen = ds.gen_vols_batch(['labeled_valid'], batch_size=1, randomize=False, load_segs=True)

min_h = 160

seg_oh_warp_model = transform_network_utils.warp_model(
				img_shape=seg_exp.aug_img_shape[:-1] + (seg_exp.n_labels,),#(1,),
				interp_mode='linear'
			)

seg_warp_model = transform_network_utils.warp_model(
				img_shape=seg_exp.aug_img_shape[:-1] + (1,),
				interp_mode='nearest'
			)

dices_supflow_vs_true = np.zeros((n_examples * 2, seg_exp.n_labels))
dices_augflow_vs_true = np.zeros((n_examples * 2, seg_exp.n_labels))
dices_suppred_vs_true = np.zeros((n_examples * 2, seg_exp.n_labels))
dices_augpred_vs_true = np.zeros((n_examples * 2, seg_exp.n_labels))
dices_warpedoh_vs_true = np.zeros((n_examples * 2, seg_exp.n_labels))
dices_warped_vs_true = np.zeros((n_examples * 2, seg_exp.n_labels))


sup_atlas_segs = segmenter_networks.segment_vol_by_slice(seg_sup_model, source_X, seg_exp.label_mapping)
aug_atlas_segs = segmenter_networks.segment_vol_by_slice(seg_exp.segmenter_model, source_X, seg_exp.label_mapping)
    
dice_supatlas = medipy_metrics.dice(source_Y, sup_atlas_segs, labels=seg_exp.label_mapping)[0, :]
dice_augatlas = medipy_metrics.dice(source_Y, aug_atlas_segs, labels=seg_exp.label_mapping)[0, :]
print('Supseg_model atlas vs true atlas segs dice: {}'.format(np.mean(dice_supatlas[1:])))
print('Augseg_model atlas vs true atlas segs dice: {}'.format(np.mean(dice_augatlas[1:])))
    
for ei in range(n_examples, n_examples * 2):
    if ei == 0:
        print('Training targets')

    elif ei == n_examples:
        print('Test targets')
        
    if ei < n_examples:
        target_X, target_Y = next(train_vol_gen)
    else:
        target_X, target_Y = next(test_vol_gen)
        
    
    source_Y_oh = classification_utils.labels_to_onehot(source_Y, label_mapping=seg_exp.label_mapping)
   
    slice_idx = 112
    
    preds = vm_model.predict([source_X, target_X])

    # parse predicted volumes
    if len(preds) > 2:
        flow = preds[0]
        warped = preds[1]
        color_delta = preds[2]
        transformed = preds[3]
    else:
        flow = preds[0]
        warped = preds[1]
        transformed = warped

        
    # warp segmentations according to voxelmorph flow, from onehot representations
    warped_segs_oh = seg_oh_warp_model.predict([source_Y_oh, flow])
    warped_segs_oh = classification_utils.onehot_to_labels(warped_segs_oh, label_mapping=seg_exp.label_mapping)
    
    # try warping label maps
    warped_segs = seg_warp_model.predict([source_Y[...,np.newaxis], flow])
    
    # now try to run the segmenter on the synthetic volume

    sup_flow_segs = segmenter_networks.segment_vol_by_slice(seg_sup_model, warped, seg_exp.label_mapping)
    aug_flow_segs = segmenter_networks.segment_vol_by_slice(seg_exp.segmenter_model, warped, seg_exp.label_mapping)
    sup_pred_segs = segmenter_networks.segment_vol_by_slice(seg_sup_model, transformed, seg_exp.label_mapping)
    aug_pred_segs = segmenter_networks.segment_vol_by_slice(seg_exp.segmenter_model, transformed, seg_exp.label_mapping)

    dice_supflow_vs_true = medipy_metrics.dice(target_Y, sup_flow_segs, labels=seg_exp.label_mapping)[0, :]
    dice_augflow_vs_true = medipy_metrics.dice(target_Y, aug_flow_segs, labels=seg_exp.label_mapping)[0, :]
    dice_suppred_vs_true = medipy_metrics.dice(target_Y, sup_pred_segs, labels=seg_exp.label_mapping)[0, :]
    dice_warpedoh_vs_true = medipy_metrics.dice(target_Y, warped_segs_oh, labels=seg_exp.label_mapping)[0, :]
    dice_warped_vs_true = medipy_metrics.dice(target_Y, warped_segs, labels=seg_exp.label_mapping)[0, :]
    dice_augpred_vs_true = medipy_metrics.dice(target_Y, aug_pred_segs, labels=seg_exp.label_mapping)[0, :]
    
    dices_supflow_vs_true[ei] = dice_supflow_vs_true
    dices_augflow_vs_true[ei] = dice_augflow_vs_true
    dices_suppred_vs_true[ei] = dice_suppred_vs_true
    dices_warpedoh_vs_true[ei] = dice_warpedoh_vs_true
    dices_warped_vs_true[ei] = dice_warped_vs_true
    dices_augpred_vs_true[ei] = dice_augpred_vs_true

    # draw flow using the same function as voxelmorph paper
    flow_im, _, _ = vis_utils.flow_to_im(flow[0, :, :, slice_idx], clip_flow=20)
    out_ims = [
        vis_utils.label_ims(source_X[:, :, :, slice_idx], 'atlas', display_h=min_h),
        flow_im,
        vis_utils.label_ims(warped[:, :, :, slice_idx], 'warped', display_h=min_h),
    ]
    if len(preds) > 2:
        out_ims += [
            vis_utils.label_ims(color_delta[:, :, :, slice_idx], 'color_delta', normalize=True, display_h=min_h),
            vis_utils.label_ims(transformed[:, :, :, slice_idx], 'transformed', display_h=min_h),
        ]
    

    out_ims += [vis_utils.label_ims(target_X[:, :, :, slice_idx], 'tgt_subject', display_h=min_h)]
    out_im = np.concatenate(out_ims, axis=1)
    IPython.display.display(PIL.Image.fromarray((out_im[..., [2, 0, 1]]).astype(np.uint8)))
    
    print('Onehotwarped vs true segs dice: {}'.format(np.mean(dice_warpedoh_vs_true[1:])))
    print('Warped vs true segs dice: {}'.format(np.mean(dice_warped_vs_true[1:])))
    print('Supseg_model flow-only pred vs true segs dice: {}'.format(np.mean(dice_supflow_vs_true[1:])))
    print('Augseg_model flow-only pred vs true segs dice: {}'.format(np.mean(dice_augflow_vs_true[1:])))
    print('Supseg_model pred vs true segs dice: {}'.format(np.mean(dice_suppred_vs_true[1:])))
    print('Augseg_model pred vs true segs dice: {}'.format(np.mean(dice_augpred_vs_true[1:])))
    
    if ei == n_examples - 1:
        print('')     
        print('Total Onehotwarped vs true segs dice: {}'.format(np.mean(dices_warpedoh_vs_true[:n_examples, 1:])))
        print('Total Warped vs true segs dice: {}'.format(np.mean(dices_warped_vs_true[:n_examples, 1:])))
        print('Supseg_model flow-only pred vs true segs dice: {}'.format(np.mean(dices_supflow_vs_true[1:])))
        print('Augseg_model flow-only pred vs true segs dice: {}'.format(np.mean(dices_augflow_vs_true[1:])))
        print('Total Supseg_model pred vs true segs dice: {}'.format(np.mean(dices_suppred_vs_true[:n_examples, 1:])))
        print('Total Augseg_model pred vs true segs dice: {}'.format(np.mean(dices_augpred_vs_true[:n_examples, 1:])))
print('')     
print('Total Onehotwarped vs true segs dice: {}'.format(np.mean(dices_warpedoh_vs_true[n_examples:, 1:])))
print('Total Warped vs true segs dice: {}'.format(np.mean(dices_warped_vs_true[n_examples:, 1:])))
print('Supseg_model flow-only pred vs true segs dice: {}'.format(np.mean(dices_supflow_vs_true[1:])))
print('Augseg_model flow-only pred vs true segs dice: {}'.format(np.mean(dices_augflow_vs_true[1:])))
print('Total Supseg_model pred vs true segs dice: {}'.format(np.mean(dices_suppred_vs_true[n_examples:, 1:])))
print('Total Augseg_model pred vs true segs dice: {}'.format(np.mean(dices_augpred_vs_true[n_examples:, 1:])))

In [ ]:
subj_idx = 0

test_vol_gen = ds.gen_vols_batch(['labeled_valid'], batch_size=1, randomize=False, load_segs=True)

for i in range(subj_idx + 1):
    target_X, target_Y = next(test_vol_gen)

IPython.display.display(PIL.Image.fromarray((target_X[0, :, :, 112, 0] * 255).astype(np.uint8)))

from dataset_utils import adni_loader
X, Y = adni_loader._load_vol_and_seg(ds.files_labeled_valid[subj_idx], mask_vol=True)
print(X.shape)
IPython.display.display(PIL.Image.fromarray((X[:, :, 112, 0] * 255).astype(np.uint8)))
